# 数据获取

## 基本原则
数据来源于微博。由于微博用户较大，若随机获取用户很难构成网络，基于此，一共3500个用户的关联网络获取思路为：从一个用户出发，获取到他所有双向关注用户的列表后，再获取这些用户的双向关注的用户列表。最终获取到3500多名用户的列表。

## 代码改进
1. 由于cookie是动态的，需要常常重新获取cookie
2. 从“散装”代码进步为用class和def封装
3. try——except 保证代码顺利进行

In [ ]:
import pandas as pd
import json
data = pd.read_csv('list_toget2.csv',encoding = 'utf-8')
list1 = data['id'][1590:]
#len(data)

In [2]:
import requests
import time
import math

In [ ]:
def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Referer": "https://weibo.com"
    }
    cookies = {
        'cookie':'SINAGLOBAL=4954692280506.27.1591184066784; UOR=,,www.baidu.com; ULV=1644499064874:67:1:1:326185633315.2391.1644499064716:1641877820203; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFd.fwHbnebHoYw4M0SwxAm5JpX5KMhUgL.Foe7eoqXeoBceh-2dJLoI7__UgSQIs8DMJxRwntt; ALF=1676902879; SSOLoginState=1645366880; SCF=Au_QDngCAtZdjo2U3MOWDe9AqNjD2bv9bCKkhre3FM48msa0qRN3YSgS14rkF-AbZ4AmYHQ9T9Yb_JlBWsV8wHw.; SUB=_2A25PFj4wDeRhGeVO6VQV8irKyzmIHXVsYij4rDV8PUNbmtB-LXHNkW9NTWqL0EtXCJXhYASgZqyGjQjjIpN7p0MS; XSRF-TOKEN=so0_TtOjvPYlcYWND-NcBS_d; WBPSESS=jFjxfnnr6hQgrI4dXoznc7bnqdlyUe9Js9XErv9qzzQHTfxCJsjK1-ncjVFLlj4u5cCl5-VQV_f8SBNITtVimIMp7mv5JIOpbg1G0Tf1h_e8ds8wvsWW_TbJlnuTE7l5'
        #'cookie':'SINAGLOBAL=4954692280506.27.1591184066784; UOR=,,www.baidu.com; ULV=1640670361949:63:14:1:8632450552135.697.1640670361940:1639835996800; SCF=Au_QDngCAtZdjo2U3MOWDe9AqNjD2bv9bCKkhre3FM48Gh6mDFxMCacqDmefa19z_7fYJ9W_ldjCbGt1vKO7CaI.; SUB=_2A25Mz7rVDeRhGeVO6VQV8irKyzmIHXVvvKsdrDV8PUJbmtAKLRn-kW9NTWqL0FkKERyR5ZWAAhKgqSgGF5QPtnkp; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFd.fwHbnebHoYw4M0SwxAm5JpX5K-hUgL.Foe7eoqXeoBceh-2dJLoI7__UgSQIs8DMJxRwntt; ALF=1672280760; SSOLoginState=1640745605; XSRF-TOKEN=8rdPjDQ-y5jyz0mWRw5ZLY24'
    }
    response = requests.get(url, headers=headers, cookies=cookies)
    time.sleep(3)   # 加上5s 的延时防止被反爬
    return response.text
def get_fans_data(uid):
    # 先获取总的粉丝数量
    url = "https://www.weibo.com/ajax/friendships/friends?relate=fans&uid=" + str(uid) 
    html = get_html(url)
    url2 = "https://www.weibo.com/ajax/friendships/friends?relate=followers&uid=" + str(uid) + "&page=3"
    html2 = get_html(url2)
    try:
        response2 = json.loads(html2)
        response = json.loads(html)
        
        total_number = response['total_number']
        total_number2 = response2['total_number']
        lista = []
        listb = []
        # 然后依次爬取每一页的数据
        for page in range(1, math.ceil(total_number/20) + 1):
            url = "https://www.weibo.com/ajax/friendships/friends?relate=fans&uid=" + str(uid) + "&page=" + str(page)
            html = get_html(url)
            response = json.loads(html)
            if response.__contains__("users") == False:
                break
            else:
                fans_list = response['users']
                data = {}   # 创建一个字典存放数据
                for fan in fans_list:
                    #data['uid'] = uid
                    data['id'] = fan['id']     # 用户id
                    data['screen_name'] = fan['screen_name']    # 用户昵称
                    data['description'] = fan['description']  # 个性签名
        #             data['gender'] = fan['gender']  # 性别
        #             data['followers_count'] = fan['followers_count']    # 粉丝的粉丝数量
        #             data['friends_count'] = fan['friends_count']    # 粉丝的关注数量
        #             data['statuses_count'] = fan['statuses_count']    # 粉丝的博文数量
                    lista.append(data)
                    data={}
                # 还有很多信息可以得到，我这里就不再举例子了


        for page in range(1, math.ceil(total_number2/20) + 1):
            url = "https://www.weibo.com/ajax/friendships/friends?relate=followers&uid=" + str(uid) + "&page=" + str(page)
            html = get_html(url)
            response2 = json.loads(html)
            #print(html)
            if response2.__contains__("users") == False:
                break
            else:
                followers_list = response2['users']
                data = {}   # 创建一个字典存放数据
                for follower in followers_list:
                    #data['uid'] = uid
                    data['id'] = follower['id']     # 用户id
                    data['screen_name'] = follower['screen_name']    # 用户昵称
                    data['description'] = follower['description']  # 个性签名
        #             data['gender'] = fan['gender']  # 性别
        #             data['followers_count'] = fan['followers_count']    # 粉丝的粉丝数量
        #             data['friends_count'] = fan['friends_count']    # 粉丝的关注数量
        #             data['statuses_count'] = fan['statuses_count']    # 粉丝的博文数量
                    listb.append(data)
                    data={}
                # 还有很多信息可以得到，我这里就不再举例子了
        data2 = {}
        data2['fans'] = lista
        data2['followers'] = listb
        #listm = []
        #listm.append(data2)
        data3 = {}
        data3[uid] = data2
        save_fans_data_json(data3,uid)
    #save_fans_data_json(lista)
    except:
        print(url+'no')
def save_fans_data_json(data,uid):
    info_json = json.dumps(data,sort_keys=False, indent=4, separators=(',', ': '))
    #data2 = 
    file_test = '0212/'+str(uid) + 'info.json'
    f = open(file_test, 'w')
    f.write(info_json)
for uid in list1:
    get_fans_data(uid)
    print(uid)

## 数据合并

- 这样获取的数据是每个用户为一个单独的json文件，为了后续读取文件的便捷以及制作网络图的便捷性，将其合并为一个文件

In [ ]:
import os
import json
path = '/Users/daigaku/Desktop/毕业论文/三月稿1/data/'
files = os.listdir(path)
data2 = {}
json_list = []

for file in files:
    file_name = file
    data = open(file_name, encoding='utf-8')

    #print(file_name)
    strJson = json.load(data)
    #print(strJson)
    data2.update(strJson)
#     #json_list.append(strJson)
#     #data2 = data2 + strJson
#     #print(strJson)
#     print(data2)
#print(data2)
#json_list

In [ ]:
for i in json_list:
    print(i)

In [ ]:
len(data2)

In [ ]:
info_json = json.dumps(data2,sort_keys=False, indent=4, separators=(',', ': '))
#data2 = 
f = open('total_data.json', 'w')
f.write(info_json)

# 最终得到total_data.json 文件（见data文件夹）